# Trabalho 2 - Dashboard

## Componentes

- Raphael Ramos
- Vitor Gabriel

In [1]:
import sys
sys.path.append('../')

import pandas as pd

data_path = '../data'
streamlit_data_path = '../streamlit/data'

df_air_quality = pd.read_csv('{}/india_air_quality_daily_2015_to_2020.csv'.format(data_path))

## Objetivo

Este notebook visa exportar os conjuntos de dados que serão utilizados para os gráficos interativos do Dashboard que será apresentado numa página web, assim como apresentar as suas implementações utilizando o pacote Plotly

## Conjuntos de dados

### AQI anual nas cidades da Índia

Média do AQI anual nas cidades presente no conjunto de dados utilizados, assim como sua classificação categórica (boa, moderada, insalubre e perigosa)

In [2]:
# Latitude e Longitude de cada cidade
city_lat_long = [
    {"_City": "Patna", "Lat": 25.5941, "Lon": 85.1376},
    {"_City": "Mumbai", "Lat": 19.0760, "Lon": 72.8777},
    {"_City": "Lucknow", "Lat": 26.8467, "Lon": 80.9462},
    {"_City": "Jaipur", "Lat": 26.9124, "Lon": 75.7873},
    {"_City": "Delhi", "Lat": 28.6139, "Lon": 77.2090},
    {"_City": "Chennai", "Lat": 13.0827, "Lon": 80.2707},
    {"_City": "Ahmedabad", "Lat": 23.0225, "Lon": 72.5714},
    {"_City": "Amritsar", "Lat": 31.6340, "Lon": 74.8723},
    {"_City": "Aizawl", "Lat": 23.7271, "Lon": 92.7176},
    {"_City": "Amaravati", "Lat": 16.5667, "Lon": 80.3667},        
    {"_City": "Bengaluru", "Lat": 12.9716, "Lon": 77.5946},        
    {"_City": "Bhopal", "Lat": 23.2599, "Lon": 77.4126},           
    {"_City": "Brajrajnagar", "Lat": 21.8167, "Lon": 83.9167},     
    {"_City": "Chandigarh", "Lat": 30.7415, "Lon": 76.7681},       
    {"_City": "Coimbatore", "Lat": 11.0046, "Lon": 76.9616},       
    {"_City": "Ernakulam", "Lat": 9.9800, "Lon": 76.2800},         
    {"_City": "Gurugram", "Lat": 28.4595, "Lon": 77.0266},         
    {"_City": "Guwahati", "Lat": 26.1445, "Lon": 91.7362},         
    {"_City": "Hyderabad", "Lat": 17.3850, "Lon": 78.4867},        
    {"_City": "Jorapokhar", "Lat": 23.5000, "Lon": 86.1833},       
    {"_City": "Kochi", "Lat": 9.9312, "Lon": 76.2673},             
    {"_City": "Shillong", "Lat": 25.5788, "Lon": 91.8933},         
    {"_City": "Talcher", "Lat": 20.7733, "Lon": 85.1667},          
    {"_City": "Thiruvananthapuram", "Lat": 8.5241, "Lon": 76.9366},
    {"_City": "Visakhapatnam", "Lat": 17.6868, "Lon": 83.2185},    

]

# Função para classificar o AQI
def classify_aqi(aqi):
    if aqi <= 50:
        return "Good"
    elif aqi <= 100:
        return "Moderate"
    elif aqi <= 150:
        return "Poor"
    elif aqi <= 200:
        return "Unhealthy"
    elif aqi <= 300:
        return "Severe"
    else:
        return "Hazardous"

df_cities_aqi = df_air_quality[['City', 'Date', 'AQI']].copy()

# Remover linhas com pelo menos uma coluna null
cities_aqi_filtered = df_cities_aqi.dropna().copy()

# Converter a coluna 'Date' para datetime e extrair o ano
cities_aqi_filtered['Date'] = pd.to_datetime(cities_aqi_filtered['Date'])
cities_aqi_filtered['Year'] = cities_aqi_filtered['Date'].dt.year

# Colocar o AQI médio anual de cada cidade
annual_aqi = cities_aqi_filtered.groupby(['City', 'Year'])['AQI'].mean().reset_index()
annual_aqi['AQI_Category'] = annual_aqi['AQI'].apply(lambda x : classify_aqi(x))

# Converte city_lat_long para DataFrame
city_coords_df = pd.DataFrame(city_lat_long)

# Faz o merge com base na coluna de cidade
annual_aqi = annual_aqi.merge(city_coords_df, how='left', left_on='City', right_on='_City')
annual_aqi = annual_aqi.drop(columns=['_City'])

# Exportar dataframe para CSV
annual_aqi.to_csv('{}/dashboard_annual_aqi.csv'.format(streamlit_data_path))

annual_aqi

,City,Year,AQI,AQI_Category,Lat,Lon
0,Ahmedabad,2015,310.950570,Hazardous,23.0225,72.5714
1,Ahmedabad,2016,310.162393,Hazardous,23.0225,72.5714
2,Ahmedabad,2017,558.768116,Hazardous,23.0225,72.5714
3,Ahmedabad,2018,622.263305,Hazardous,23.0225,72.5714
4,Ahmedabad,2019,516.352273,Hazardous,23.0225,72.5714
...,...,...,...,...,...,...
93,Visakhapatnam,2016,103.976048,Poor,17.6868,83.2185
94,Visakhapatnam,2017,143.094340,Poor,17.6868,83.2185
95,Visakhapatnam,2018,122.819018,Poor,17.6868,83.2185
96,Visakhapatnam,2019,123.442815,Poor,17.6868,83.2185


### AQI *versus* poluentes PM2.5 e PM10

Média mensal dos poluentes PM10, PM2.5 e do AQI ao no periodo 2015-2020 para atestar a correlação entre eles

In [3]:
df_pollutants_aqi = df_air_quality[['City', 'Date', 'PM10', 'PM2.5', 'AQI']]

# Remover linhas com pelo menos uma coluna null
pollutants_aqi_filtered = df_pollutants_aqi.dropna().copy()

# Converter a coluna 'Date' para datetime e extrair o ano e mês
pollutants_aqi_filtered['Date'] = pd.to_datetime(pollutants_aqi_filtered['Date'])
pollutants_aqi_filtered['YearMonth'] = (
    pollutants_aqi_filtered['Date'].dt.year.astype(str) + '-' +
    pollutants_aqi_filtered['Date'].dt.month.astype(str).str.zfill(2)
)

# Valor médio de concentrações de partículas e AQI mensal, por cidade
month_aqi_pm10_pm25 = pollutants_aqi_filtered.groupby(
    by=['City', 'YearMonth']
)[['PM10', 'PM2.5', 'AQI']].mean().reset_index()

# ===============================================
# ✅ Lógica adicional: preencher meses ausentes
# ===============================================

# Cidades e meses únicos
cities = month_aqi_pm10_pm25['City'].unique()
months = sorted(month_aqi_pm10_pm25['YearMonth'].unique())

# Produto cartesiano entre cidades e meses
full_index = pd.MultiIndex.from_product([cities, months], names=["City", "YearMonth"])
full_df = pd.DataFrame(index=full_index).reset_index()

# Merge com os dados já agrupados
month_aqi_pm10_pm25 = pd.merge(full_df, month_aqi_pm10_pm25, on=["City", "YearMonth"], how="left")

# Preencher valores ausentes com zero
month_aqi_pm10_pm25['PM10'] = month_aqi_pm10_pm25['PM10'].fillna(0)
month_aqi_pm10_pm25['PM2.5'] = month_aqi_pm10_pm25['PM2.5'].fillna(0)
month_aqi_pm10_pm25['AQI'] = month_aqi_pm10_pm25['AQI'].fillna(0)

# Ordenar por ano-mês e cidade
month_aqi_pm10_pm25.sort_values(by=['YearMonth', 'City'], inplace=True)

# Exportar dataframe para CSV
month_aqi_pm10_pm25.to_csv(f'{streamlit_data_path}/dashboard_month_aqi_pm10_pm25.csv', index=False)
